In [1]:
!pip install tensorflow pandas numpy matplotlib nltk scikit-learn gensim


In [2]:
!pip install keras

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input,GlobalMaxPool1D,Dropout
from keras.utils import plot_model

import nltk
import re
from nltk.corpus import stopwords #corpus is collection of text
from nltk.stem.porter import PorterStemmer

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, roc_auc_score

In [2]:
# load train data
train = pd.read_csv('D:\\Google\\Models\\fakeNewsDataset\\train.csv')
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
#drop the Nan Values
train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
# replace na
train['text'].fillna( train['title'],inplace=True)
train.isnull().sum()

id           0
title      558
author    1957
text         0
label        0
dtype: int64

In [8]:
#Get the Depndent feature
X_train=train.drop('label',axis=1)
y_train=train['label']

In [9]:
# set vocabulary size
vo_size=500
messages=X_train.copy()
messages.reset_index(inplace=True)

In [10]:
# Preprocess the text data
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords.words('english')]
    return ' '.join(text)

train['processed_text'] = train['text'].apply(preprocess_text)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:

# Set vocabulary size
vo_size = 500
DIM=100

# Tokenize the corpus
sentences_list = [text.split() for textt in train['processed_text'].tolist()]

# Train Word2Vec model using the tokenized corpus
word2vec_model = Word2Vec(sentences=sentences_list, vector_size=DIM, window=10, min_count=1)


tokenizer=Tokenizer()
tokenizer.fit_on_texts(sentences_list)
x=tokenizer.texts_to_sequences(sentences_list)


# Convert the entire corpus into list of embeddings

embedded_doc = []
for sequence in x:
    embedded_sequence = []
    for word_index in sequence:
        word = tokenizer.index_word[word_index]
        if word in word2vec_model.wv:
            embedded_sequence.append(word2vec_model.wv[word])
    embedded_doc.append(embedded_sequence)
# Prepare X_final and y_final for Neural Network
embedded_doc = pad_sequences(embedded_doc, padding='pre', maxlen=1000)
X_final = np.array(embedded_doc)
y_final = np.array(y_train)


vocab_size=len(tokenizer.word_index)+1
vocab=tokenizer.word_index
# Model Building
embedding_vector_feature = 100
model = Sequential()

# Add embedding layer with Word2Vec weights
model.add(Embedding(input_dim=len(word2vec_model.wv.index_to_key), output_dim=embedding_vector_feature, weights=[word2vec_model.wv.vectors], input_length=1000, trainable=False))

# Add LSTM layer
model.add(LSTM(200))

# Add Dense layer
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Plot Model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

# Train Model
history = model.fit(X_final, y_final, validation_split=0.2, epochs=10, batch_size=256)



In [13]:
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
# Train Word2Vec model
corpus = [text.split() for text in train['processed_text']]
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1)

# Convert text data to sequences of Word2Vec embeddings and pad sequences
max_sequence_length = 1000
X_train_word2vec = []
for text in corpus:
    word2vec_sequence = []
    for word in text:
        if word in word2vec_model.wv:
            word2vec_sequence.append(word2vec_model.wv[word])
    X_train_word2vec.append(word2vec_sequence)

X_train_word2vec = pad_sequences(X_train_word2vec, padding='pre', maxlen=max_sequence_length)

# Prepare target variable
y_train = train['label']

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_word2vec, y_train, test_size=0.2, random_state=42)






In [24]:
print(len(word2vec_model.wv.key_to_index))

136522


In [26]:
print(word2vec_model.wv.vector_size)

100


In [36]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
import numpy as np

# Assuming you have already trained the Word2Vec model (word2vec_model) and defined max_sequence_length

# Get the vocabulary size and vector dimension
vocab_size = len(word2vec_model.wv.key_to_index)+1
vector_dim = 100

# Build the Sequential model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=vector_dim, trainable=False))



model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [37]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=256)



MemoryError: Unable to allocate 6.20 GiB for an array with shape (16640, 1000, 100) and data type int32

In [ ]:
# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Model Accuracy')
plt.legend()
plt.show()



In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/modelWeights/fake_news_classifier.h5')

In [11]:
# Embed all words in the entire corpus
all_words = [word for sentence in corpus for word in sentence]
word_embeddings = [word2vec_model.wv[word] if word in word2vec_model.wv else np.zeros(100) for word in all_words]


from gensim.models import Word2Vec
word2vec_model = Word2Vec(sentences=corpus, vector_size=100, window=5, min_count=1)

# Get Word2Vec embeddings
X_train_word2vec = []
for sentence in corpus:
    word2vec_sentence = []
    for word in sentence:
        if word in word2vec_model.wv.key_to_index:
            word2vec_sentence.append(word2vec_model.wv[word])
    X_train_word2vec.append(word2vec_sentence)

# Pad sequences
sent_length = 1000
X_train_word2vec = pad_sequences(X_train_word2vec, padding='pre', maxlen=sent_length)


In [12]:
# model 1 build
model = Sequential()
model.add(Embedding(input_dim=len(word2vec_model.wv), 
                    output_dim=word2vec_model.vector_size, 
                    weights=[word2vec_model.wv.vectors], 
                    trainable=False))
model.add(LSTM(200))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())



KeyboardInterrupt



In [ ]:
# Plot title model
plot_model(model, to_file='model_plot2.png', show_shapes=True, show_layer_names=True)

In [ ]:
# check shape
len(embedded_doc),y_train.shape

In [ ]:
# final data for NN
X_final=np.array(embedded_doc)
y_final=np.array(y_train)
X_final.shape,y_final.shape

In [ ]:
# train model 1
history = model.fit(X_final,y_final, validation_split=0.2, epochs=10, batch_size=256)

In [ ]:
# summarize history for accuracy

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('/content/drive/MyDrive/modelWeights/stm1.h5')

In [ ]:
# model 2 build
model = Sequential()
model.add(Embedding(vo_size,embedding_vector_feature,input_length=sent_length))
model.add(LSTM(100, return_sequences=True,name='lstm_layer'))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(50, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Plot title model
plot_model(model, to_file='model_plot3.png', show_shapes=True, show_layer_names=True)

In [ ]:
# train model 2
history = model.fit(X_final,y_final, validation_split=0.2, epochs=10, batch_size=256)

In [ ]:
# summarize history for accuracy

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# load test
test = pd.read_csv('Data\\test.csv')
test.head()

In [ ]:
# check na in test
test.isnull().sum()

In [ ]:
# Replace na
test['text'] = test['text'].replace(np.nan, test['title'])
test.isnull().sum()

In [ ]:
# prepare test data for NN
X_test=test
messages=X_test.copy()
messages.reset_index(inplace=True)
ps =PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print("Status: %s / %s" %(i, len(messages)), end="\r")
    review = re.sub('[^a-zA-Z]', ' ',messages['text'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)
onehot_rep = [one_hot(words, vo_size) for words in corpus]
embedded_doc=pad_sequences(onehot_rep, padding='pre', maxlen=sent_length)
X_test_final=np.array(embedded_doc)

In [ ]:
# predict final
y_pred_final=model.predict_classes(X_test_final)
y_pred_final = pd.DataFrame(y_pred_final)
submit = pd.concat([test['id'].reset_index(drop=True), y_pred_final], axis=1)
submit.rename(columns={ submit.columns[1]: "label" }, inplace = True)
submit.to_csv('submit_2.csv', index=False)

In [ ]:
# Save model
model.save_weights("model_text.h5")